In [11]:
import datetime

In [1]:
import ee

try:
    # Initialize the library.
    ee.Initialize()
except:
    # Trigger the authentication flow.
    ee.Authenticate()
    # Initialize the library.
    ee.Initialize()

In [48]:
aoi = ee.FeatureCollection({
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.320770263671875,
              49.669405434570464
            ],
            [
              8.683319091796875,
              49.669405434570464
            ],
            [
              8.683319091796875,
              49.99714673955337
            ],
            [
              8.320770263671875,
              49.99714673955337
            ],
            [
              8.320770263671875,
              49.669405434570464
            ]
          ]
        ]
      }
    }
  ]
})

In [49]:
bandNamesOut_s2_sr = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','vater vapor','swir1','swir2','QA60']
bandNamesS2_SR  = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12','QA60']
s2_sr = ee.ImageCollection('COPERNICUS/S2_SR').select(bandNamesS2_SR,bandNamesOut_s2_sr).filterDate(datetime.datetime(2017, 4, 1), datetime.datetime(2017, 8, 3)).filterBounds(aoi).map(maskS2clouds).map(NDVI)#.map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_

In [50]:
mosaick = ee.Image(s2_sr.qualityMosaic('NDVI')).select('NDVI')

In [71]:
mosaick = ee.Image(s2_sr.select('NDVI').mosaic())

In [ ]:
def mask_by_landcover(image):
    image = image.select('Map')
    mask = image.eq(40).Or(image.eq(30))
    return image.updateMask(mask)
#Mask out area with invalid landcover class like build up areas
lc = ee.Image(ee.ImageCollection("ESA/WorldCover/v100").first().clip(fcg_aoi.union().geometry()).select('Map'))
icg_s1 = icg_s1.map(lambda x: x.addBands(lc))
icg_s1 = icg_s1.map(mask_by_landcover)
    

In [73]:
#Segmentation of ndvi values 
segmented = ee.Algorithms.Image.Segmentation.SNIC(mosaick)

In [74]:
segmented.getInfo()

{'type': 'Image',
 'bands': [{'id': 'clusters',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'NDVI_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'seeds',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [77]:
from IPython.display import Image

segmentedV = segmented.select('clusters').clip(aoi)

# Create the URL associated with the styled image data.
url = segmentedV.getThumbUrl({
    'min': -3,'max': 40, 'region': aoi.union().geometry(), 'dimensions': 512})

# Display a thumbnail of elevation in France.
Image(url=url)


In [72]:
from IPython.display import Image

mosaick = mosaick.clip(aoi)

# Create the URL associated with the styled image data.
url = mosaick.getThumbUrl({
    'min': -1,'max': 1, 'region': aoi.union().geometry(), 'dimensions': 512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display a thumbnail of elevation in France.
Image(url=url)


In [56]:
segmented.getInfo()

{'type': 'Image',
 'bands': [{'id': 'clusters',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'NDVI_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'seeds',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [7]:
#From erath engine 01

def get_geometry(ImageCollection):
    geometry = ImageCollection.geometry().getInfo()
    #geometry = [x for x in geometry['coordinates']]
    return geometry

def clip_aoi(ImageCollection):
    im = ImageCollection.clip(ried_225_222)
    return im

def show_tiles(ImageCollection):
    geometry = get_geometry(ImageCollection)
    geometry_list = list(geometry['coordinates'])
    flattened_list = [y for x in geometry_list for y in x] #De flatter list 
    unique_list = []
    unique_list = [x for x in flattened_list if x not in unique_list]
    print("different tiles: ",len(unique_list))
    m = Map(center=(49.6252978589571, 8.34580993652344), zoom=7)
    geo_json = GeoJSON(data=geometry, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
    m.add_layer(geo_json)
    dc = ipyleaflet.DrawControl()
    m.add_control(dc)
    m.add_control(LayersControl())
    return m
    
def get_properties_l(ImageCollection):
    features = ImageCollection.getInfo()['features']
    dict_list = []
    for f in features:
        prop = f['properties']
        dict_list.append(prop)
    df = pd.DataFrame.from_records(dict_list).drop(['system:footprint'],axis=1)
    #Pandas Series of unique distinc values in df
    unique = df.nunique()
    im_id_list = [item.get('id') for item in ImageCollection.getInfo().get('features')]
    date_list = [datetime.datetime.strptime(x[36:44],'%Y%m%d') for x in im_id_list]
    #property_names = list(df.columns.values) 
    return unique, im_id_list, date_list

def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)
#A function to mask out cloudy pixels.

def cloudMaskL457(image):
    qa = image.select('pixel_qa');
    #If the cloud bit (5) is set and the cloud confidence (7) is high
    #or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    #Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min());
    return image.updateMask(cloud.Not()).updateMask(mask2)

def maskL8sr(image):
    #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5);
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

def maskQuality(image):
    # Select the QA band.
    QA = image.select('pixel_qa')
    # Get the internal_cloud_algorithm_flag bit.
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    #  var cloud_confidence = getQABits(QA,6,7,  'cloud_confidence')
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    #var cirrus_detected2 = getQABits(QA,8,8,  'cirrus_detected2')
    #Return an image masking out cloudy areas.
    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0)))

def maskS2clouds(image):
    qa = image.select('QA60')
    #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

#Function to mask out every NDVi <0.1 (Water) 
def maskNDVI(image):                   
    ndvi = image.select('NDVI')
    ndvi_mask = ndvi.gte(0.8).And(ndvi.lte(0.1)) 
    return image.updateMask(ndvi_mask)


#https://de.wikipedia.org/wiki/Normalized_Difference_Vegetation_Index
#https://www.linkedin.com/pulse/ndvi-ndbi-ndwi-calculation-using-landsat-7-8-tek-bahadur-kshetri


def NDVI(image):
    ndvi = image.normalizedDifference(['nir','red']) #(first − second) / (first + second)
    return image.addBands(ndvi).rename(image.bandNames().add('NDVI'))

def SAVI(image): #https://de.wikipedia.org/wiki/Soil-Adjusted_Vegetation_Index bodenbereinigter Vegetationsindex (BBVI) 
    #A function to compute Soil Adjusted Vegetation Index."""
    L = 0.5
    savi = ee.Image(0).expression('(1 + L) * float(nir - red)/ (nir + red + L)', {'nir': image.select('nir'), 'red': image.select('red'), 'L': 0.5})
    return image.addBands(savi).rename(image.bandNames().add('SAVI'))

#the normalized difference bare index
def NDBI_swir1(image):
    ndbi = image.normalizedDifference(['swir1', 'nir'])
    return image.addBands(ndbi).rename(image.bandNames().add('NDBI_swir1'))

def NDBI_swir2(image):
    ndbi = image.normalizedDifference(['swir2', 'nir'])
    return image.addBands(ndbi).rename(image.bandNames().add('NDBI_swir2'))

#Built Up Index
def build_up_index(image):
    bu = image.select('NDBI_swir1').subtract(image.select('NDVI'))
    return image.addBands(bu).rename(image.bandNames().add('built_up_index_swir1'))
                                       
                                       
#the normalize difference water index
def ndwi(image):
    return image.normalizedDifference(['B3', 'B8'])


#define thresholds
bareThreshold = -0.32
vegetationThreshold = 0.65
waterThreshold = 0.2

"""
NDVI = -1 to 0 represent Water bodies
NDVI = -0.1 to 0.1 represent Barren rocks, sand, or snow
NDVI = 0.2 to 0.5 represent Shrubs and grasslands or senescing crops
NDVI = 0.6 to 1.0 represent Dense vegetation or tropical rainforest
"""


bandNamesOut_l7 = ['blue','green','red','nir','swir1','temp1','swir2','pixel_qa']
bandNamesOut_l8 = ['ultra blue','blue','green','red','nir','swir1','swir2','temp1','temp2','pixel_qa']
bandNamesOut_s2 = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','water vapor','cirrus','swir1','swir2','QA60']
bandNamesOut_s2_sr = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','vater vapor','swir1','swir2','QA60']

bandNamesl7 = ['B1','B2','B3','B4','B5','B6','B7','pixel_qa']
bandNamesl8 = ['B1','B2','B3','B4','B5','B6','B7','B10','B11','pixel_qa']
bandNamesS2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','QA60']
bandNamesS2_SR  = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12','QA60']

# Filter the L7 collection to a single month.
l7_sr = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').select(bandNamesl7,bandNamesOut_l7).filterDate(datetime.datetime(2009, 1, 1), datetime.datetime(2019, 6, 1)).filterBounds(ried_225_222).map(clip_aoi).map(cloudMaskL457).map(NDVI).map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_SR
l8_sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').select(bandNamesl8,bandNamesOut_l8).filterDate(datetime.datetime(2009, 1, 1), datetime.datetime(2019, 6, 1)).filterBounds(ried_225_222).map(clip_aoi).map(maskL8sr).map(NDVI).map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR
s2_1c = ee.ImageCollection('COPERNICUS/S2').select(bandNamesS2,bandNamesOut_s2).filterDate(datetime.datetime(2015, 6, 22), datetime.datetime(2017, 3, 27)).filterBounds(ried_225_222).map(clip_aoi).map(maskS2clouds).map(NDVI).map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2
s2_sr = ee.ImageCollection('COPERNICUS/S2_SR').select(bandNamesS2_SR,bandNamesOut_s2_sr).filterDate(datetime.datetime(2017, 3, 27), datetime.datetime(2019, 6, 1)).filterBounds(ried_225_222).map(clip_aoi).map(maskS2clouds).map(NDVI).map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_

NameError: name 'datetime' is not defined